In [0]:
! pip install soynlp 

     |████████████████████████████████| 419kB 2.9MB/s 


In [0]:
import time
import os
import pickle
import pandas as pd
import warnings
import numpy as np
import seaborn as sns
from soynlp.tokenizer import RegexTokenizer, LTokenizer, MaxScoreTokenizer
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd/content/drive/My Drive/tobigs_chatbot/

/content/drive/My Drive/tobigs_chatbot


In [0]:
df = pd.read_pickle("data.pickle") # pos 태깅된 데이터 
df.head()

,Question,Answer,pre_Question,pre_Answer,Q_token,A_token,len_Q_token,len_A_token,Question2
0,중증장애아동수당을 받는 중증장애인이 만21세가 된 경우 장애인연금을 별도로 신청해야...,중증장애아동수당을 받는 중증장애인이 초중등교육법 제2조에 따른 학교에 재학중인 경우...,중증장애아동수당을 받는 중증장애인이 만21세가 된 경우 장애인연금을 별도로 신청해야...,중증장애아동수당을 받는 중증장애인이 초중등교육법 제2조에 따른 학교에 재학중인 경우...,"[중증, 장애, 아, 동, 수당, 을, 받는, 중, 증장, 애인, 이, 만, 21,...","[중증, 장애, 아, 동, 수당, 을, 받는, 중, 증장, 애인, 이, 초, 중등교...",25,65,"[(중증, Noun), (장애, Noun), (아, Josa), (동, Modifi..."
1,장애인활동지원 교육기관과 제공기관에 대한 정보는 어디에서 확인할 수 있나요?,장애인활동지원 홈페이지(www.ableservice.or.kr)에서 지정된 활동 지...,장애인활동지원 교육기관과 제공기관에 대한 정보는 어디에서 확인할 수 있나요,장애인활동지원 홈페이지wwwableserviceorkr에서 지정된 활동 지원기관 및...,"[장애인, 활동, 지원, 교육, 기관, 과, 제, 공, 기관, 에, 대한, 정보, ...","[장애인, 활동, 지원, 홈페이지, wwwableserviceorkr, 에서, 지정...",19,21,"[(장애인, Noun), (활동, Noun), (지원, Noun), (교육, Nou..."
2,기초연금 수급권을 포기하면 장애인연금 기초급여를 받을 수 있나요?,장애인연금법 제6조 제5항에 따라 기초연금 수급권자에게는 기초급여를 지급하지 않습니다.,기초연금 수급권을 포기하면 장애인연금 기초급여를 받을 수 있나요,장애인연금법 제6조 제5항에 따라 기초연금 수급권자에게는 기초급여를 지급하지 않습니다,"[기초, 연금, 수급, 권, 을, 포기, 하면, 장애인, 연금, 기초, 급여, 를,...","[장애인, 연금, 법, 제, 6조, 제, 5, 항, 에, 따라, 기초, 연금, 수급...",15,22,"[(기초, Noun), (연금, Noun), (수급, Noun), (권, Suffi..."
3,만19세 중증장애인이 2월에 학교를 졸업하고 장애인연금을 신청하면 2월부터 연금을 ...,"장애인연금법에 따라 2월 중에는 장애인연금을 신청할 수 없으며, 미리 신청하더라도 ...",만19세 중증장애인이 2월에 학교를 졸업하고 장애인연금을 신청하면 2월부터 연금을 ...,장애인연금법에 따라 2월 중에는 장애인연금을 신청할 수 없으며 미리 신청하더라도 3...,"[만, 19, 세, 중, 증장, 애인, 이, 2월, 에, 학교, 를, 졸업, 하고,...","[장애인, 연금, 법, 에, 따라, 2월, 중, 에는, 장애인, 연금, 을, 신청,...",25,24,"[(만, Noun), (19, Number), (세, Noun), (중, Noun)..."
4,장애인연금은 누가 받을 수 있나요?,"만18세 이상의 등록 장애인 중 장애인연금법 상 중증장애인(종전 1, 2급 및 3급...",장애인연금은 누가 받을 수 있나요,만18세 이상의 등록 장애인 중 장애인연금법 상 중증장애인종전 1 2급 및 3급 중...,"[장애인, 연금, 은, 누가, 받을, 수, 있나요]","[만, 18, 세, 이상, 의, 등록, 장애인, 중, 장애인, 연금, 법, 상, 중...",7,32,"[(장애인, Noun), (연금, Noun), (은, Josa), (누가, Noun..."


In [0]:
df['Q_token'] = [[j.replace(' ','').split('/')[0] for j in x] for x in df['Q_token']]
df['Q_token']

0        [중증, 장애, 아, 동, 수당, 을, 받는, 중, 증장, 애인, 이, 만, 21,...
1        [장애인, 활동, 지원, 교육, 기관, 과, 제, 공, 기관, 에, 대한, 정보, ...
2        [기초, 연금, 수급, 권, 을, 포기, 하면, 장애인, 연금, 기초, 급여, 를,...
3        [만, 19, 세, 중, 증장, 애인, 이, 2월, 에, 학교, 를, 졸업, 하고,...
4                             [장애인, 연금, 은, 누가, 받을, 수, 있나요]
                               ...                        
10391    [장애인, 연금, 에, 대해, 서, 궁금해, 지식인, 에, 글, 을, 올립니다, 장...
10392    [교통사고, 로, 요추, 1, 번, 방출, 성, 골절, 되어, 골반, 뼈, 이식, ...
10393    [휴, 쌍둥이, 아들, 들, 이, 둘다, 장애판정, 을, 받았네요, 2, 급, 3,...
10394    [장애인, 차량, lpg, 구매, 시, 알, 고, 싶은게, 있습니다, 1, 장애인,...
10395    [제, 가, 2013년, 에, 할머니, 장애인, 등록증, 이, 있어서, GAS, 차...
Name: Q_token, Length: 10396, dtype: object

In [0]:
df['Q_token2'] = [x for x in df['Question2']] 
df['Q_token2'] 

0        [(중증, Noun), (장애, Noun), (아, Josa), (동, Modifi...
1        [(장애인, Noun), (활동, Noun), (지원, Noun), (교육, Nou...
2        [(기초, Noun), (연금, Noun), (수급, Noun), (권, Suffi...
3        [(만, Noun), (19, Number), (세, Noun), (중, Noun)...
4        [(장애인, Noun), (연금, Noun), (은, Josa), (누가, Noun...
                               ...                        
10391    [(장애인, Noun), (연금, Noun), (에, Josa), (대해, Noun...
10392    [(교통사고, Noun), (로, Josa), (요추, Noun), (1, Numb...
10393    [(휴, Exclamation), (쌍둥이, Noun), (아들, Noun), (들...
10394    [(장애인, Noun), (차량, Noun), (lpg, Alpha), (구매, N...
10395    [(제, Noun), (가, Josa), (2013년, Number), (에, Fo...
Name: Q_token2, Length: 10396, dtype: object

In [0]:
error_index = [ ]
def window_nouns(y): 
  try:
    global index
    index = [x[0] for x in y].index('장애인') 
    windows = y[index-2:index+3] 
    global nouns 
    nouns = [x[0] for x in windows if x[1]== 'Noun'] 
    return nouns
  except:
    error_index.append(y)

In [0]:
df['Window_nouns'] = df['Question2'].apply(window_nouns) 
df.head()

,Question,Answer,pre_Question,pre_Answer,Q_token,A_token,len_Q_token,len_A_token,Question2,Q_token2,Window_nouns
0,중증장애아동수당을 받는 중증장애인이 만21세가 된 경우 장애인연금을 별도로 신청해야...,중증장애아동수당을 받는 중증장애인이 초중등교육법 제2조에 따른 학교에 재학중인 경우...,중증장애아동수당을 받는 중증장애인이 만21세가 된 경우 장애인연금을 별도로 신청해야...,중증장애아동수당을 받는 중증장애인이 초중등교육법 제2조에 따른 학교에 재학중인 경우...,"[중증, 장애, 아, 동, 수당, 을, 받는, 중, 증장, 애인, 이, 만, 21,...","[중증, 장애, 아, 동, 수당, 을, 받는, 중, 증장, 애인, 이, 초, 중등교...",25,65,"[(중증, Noun), (장애, Noun), (아, Josa), (동, Modifi...","[(중증, Noun), (장애, Noun), (아, Josa), (동, Modifi...","[경우, 장애인, 연금]"
1,장애인활동지원 교육기관과 제공기관에 대한 정보는 어디에서 확인할 수 있나요?,장애인활동지원 홈페이지(www.ableservice.or.kr)에서 지정된 활동 지...,장애인활동지원 교육기관과 제공기관에 대한 정보는 어디에서 확인할 수 있나요,장애인활동지원 홈페이지wwwableserviceorkr에서 지정된 활동 지원기관 및...,"[장애인, 활동, 지원, 교육, 기관, 과, 제, 공, 기관, 에, 대한, 정보, ...","[장애인, 활동, 지원, 홈페이지, wwwableserviceorkr, 에서, 지정...",19,21,"[(장애인, Noun), (활동, Noun), (지원, Noun), (교육, Nou...","[(장애인, Noun), (활동, Noun), (지원, Noun), (교육, Nou...",[]
2,기초연금 수급권을 포기하면 장애인연금 기초급여를 받을 수 있나요?,장애인연금법 제6조 제5항에 따라 기초연금 수급권자에게는 기초급여를 지급하지 않습니다.,기초연금 수급권을 포기하면 장애인연금 기초급여를 받을 수 있나요,장애인연금법 제6조 제5항에 따라 기초연금 수급권자에게는 기초급여를 지급하지 않습니다,"[기초, 연금, 수급, 권, 을, 포기, 하면, 장애인, 연금, 기초, 급여, 를,...","[장애인, 연금, 법, 제, 6조, 제, 5, 항, 에, 따라, 기초, 연금, 수급...",15,22,"[(기초, Noun), (연금, Noun), (수급, Noun), (권, Suffi...","[(기초, Noun), (연금, Noun), (수급, Noun), (권, Suffi...","[포기, 장애인, 연금, 기초]"
3,만19세 중증장애인이 2월에 학교를 졸업하고 장애인연금을 신청하면 2월부터 연금을 ...,"장애인연금법에 따라 2월 중에는 장애인연금을 신청할 수 없으며, 미리 신청하더라도 ...",만19세 중증장애인이 2월에 학교를 졸업하고 장애인연금을 신청하면 2월부터 연금을 ...,장애인연금법에 따라 2월 중에는 장애인연금을 신청할 수 없으며 미리 신청하더라도 3...,"[만, 19, 세, 중, 증장, 애인, 이, 2월, 에, 학교, 를, 졸업, 하고,...","[장애인, 연금, 법, 에, 따라, 2월, 중, 에는, 장애인, 연금, 을, 신청,...",25,24,"[(만, Noun), (19, Number), (세, Noun), (중, Noun)...","[(만, Noun), (19, Number), (세, Noun), (중, Noun)...","[졸업, 장애인, 연금]"
4,장애인연금은 누가 받을 수 있나요?,"만18세 이상의 등록 장애인 중 장애인연금법 상 중증장애인(종전 1, 2급 및 3급...",장애인연금은 누가 받을 수 있나요,만18세 이상의 등록 장애인 중 장애인연금법 상 중증장애인종전 1 2급 및 3급 중...,"[장애인, 연금, 은, 누가, 받을, 수, 있나요]","[만, 18, 세, 이상, 의, 등록, 장애인, 중, 장애인, 연금, 법, 상, 중...",7,32,"[(장애인, Noun), (연금, Noun), (은, Josa), (누가, Noun...","[(장애인, Noun), (연금, Noun), (은, Josa), (누가, Noun...",[]


In [0]:
with open('data_window.pickle', 'wb') as f:
    pickle.dump(df, f, pickle.HIGHEST_PROTOCOL) # 키워드 앞뒤 2개 뽑힌 데이터 

In [0]:
error_index

[[('장애', 'Noun'),
  ('수당', 'Noun'),
  ('을', 'Josa'),
  ('신청', 'Noun'),
  ('하는', 'Verb'),
  ('경우', 'Noun'),
  ('장애', 'Noun'),
  ('재심사', 'Noun'),
  ('를', 'Josa'),
  ('받아야', 'Verb'),
  ('하나요', 'Noun')],
 [('장애', 'Noun'),
  ('수당', 'Noun'),
  ('대상자', 'Noun'),
  ('는', 'Josa'),
  ('어떻게', 'Adjective'),
  ('되나요', 'Verb')],
 [('의료', 'Noun'),
  ('급', 'Suffix'),
  ('여', 'Josa'),
  ('수급', 'Noun'),
  ('자', 'Suffix'),
  ('로', 'Josa'),
  ('지체', 'Noun'),
  ('장애', 'Noun'),
  ('3', 'Number'),
  ('급', 'Noun'),
  ('입니다', 'Adjective'),
  ('후방', 'Noun'),
  ('지지', 'Noun'),
  ('워커', 'Noun'),
  ('를', 'Josa'),
  ('지원', 'Noun'),
  ('받을', 'Verb'),
  ('수', 'Noun'),
  ('있나요', 'Adjective')],
 [('의료', 'Noun'),
  ('급', 'Suffix'),
  ('여', 'Josa'),
  ('수급', 'Noun'),
  ('자', 'Suffix'),
  ('로', 'Josa'),
  ('지체', 'Noun'),
  ('장애', 'Noun'),
  ('3', 'Number'),
  ('급', 'Noun'),
  ('입니다', 'Adjective'),
  ('전', 'Modifier'),
  ('방지', 'Noun'),
  ('지', 'Josa'),
  ('워커', 'Noun'),
  ('를', 'Josa'),
  ('지원', 'Noun'),
  ('받을', 'Verb'),
